In [1]:
from google.colab import files
files.upload()

# Langkah 2: Pindahkan kaggle.json ke direktori yang sesuai
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Langkah 3: Unduh dataset dari Kaggle
# !kaggle datasets download -d PromptCloudHQ/flipkart-products
!kaggle datasets download -d PromptCloudHQ/flipkart-products
# Langkah 4: Ekstrak file zip
!unzip flipkart-products.zip

# Langkah 5: Baca file CSV menggunakan pandas
import pandas as pd
product_df = pd.read_csv('flipkart_com-ecommerce_sample.csv')
product_df.head()

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/PromptCloudHQ/flipkart-products
License(s): CC-BY-SA-4.0
 55% 3.00M/5.50M [00:00<00:00, 5.69MB/s]
100% 5.50M/5.50M [00:00<00:00, 7.95MB/s]
Archive:  flipkart-products.zip
  inflating: flipkart_com-ecommerce_sample.csv  


,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [ ]:
#preprocess dataset
#Hapus missing value (null)
#product_df.isnull().sum()
# product_df.dropna(axis=0, inplace=True)
# product_df.isnull().sum()

In [2]:
product_df['text'] = product_df['product_name'] + " " + product_df['description'] + " " + product_df['product_specifications']
product_df['text'] = product_df['text'].fillna('').astype(str)

In [ ]:
# # Menyiapkan tokenizer
# import tensorflow as tf
# from tensorflow.keras.layers import TextVectorization

# vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200) #tokenisasi
# text_ds = tf.data.Dataset.from_tensor_slices(product_df['text']).batch(128) #Mengubah kolom teks menjadi tf.data.Dataset dan membaginya menjadi batch.
# vectorizer.adapt(text_ds) #Melatih layer membangun vocabulary dan tokenisasi berdasarkan teks yang diberikan

In [3]:
pip install tensorflow transformers

In [4]:
from transformers import DistilBertTokenizer, TFDistilBertModel
import tensorflow as tf

# Load pre-trained DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [5]:
def batch_tokenize_texts(texts, batch_size=32):
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        tokenized_texts = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='tf')
        yield tokenized_texts

In [6]:
def get_embeddings(texts, batch_size=32):
    embeddings = []
    for tokenized_texts in batch_tokenize_texts(texts, batch_size=batch_size):
        # Get embeddings for the current batch
        distilbert_embeddings = distilbert_model(tokenized_texts['input_ids'], attention_mask=tokenized_texts['attention_mask'])[0]
        cls_embeddings = distilbert_embeddings[:, 0, :]  # Use the [CLS] token embeddings
        embeddings.append(cls_embeddings)

    return tf.concat(embeddings, axis=0)

# Example usage with a list of texts
texts = list(product_df['text'])  # Assuming 'text' column contains the text data
cls_embeddings = get_embeddings(texts, batch_size=32)

In [ ]:
# Transformasi teks ke token
# text_vector = vectorizer(product_df['text'])

In [7]:
from tensorflow.keras.layers import Input, Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, BatchNormalization, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


#embedding_dim = 300
input_shape = cls_embeddings.shape[1]  # 768

# Input layer
input_text = Input(shape=(input_shape,), dtype=tf.float32, name='text_input')
# input_text = Input(shape=(200,), dtype=tf.int64, name='text')
#embedding = Embedding(input_dim=40000, output_dim=embedding_dim, name='embedding')(input_text)

# x = LSTM(64, return_sequences=True)(input_text)
# x = LSTM(64, return_sequences=True)(x)
# x = LSTM(64)(x)



# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)
# x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
# x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(input_text)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
output = Dense(50, activation='relu', kernel_regularizer=l2(0.01))(x)


model = Model(inputs=input_text, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.summary()
model.save('recommender')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 768)]             0         
                                                                 
 dense (Dense)               (None, 256)               196864    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                             

In [8]:
num_products = product_df.shape[0]
y_train = np.random.rand(num_products, 50)

model.fit(cls_embeddings, y_train, epochs=30, batch_size=32, validation_split=0.3)

Epoch 1/30
438/438 [==============================] - 6s 6ms/step - loss: 0.3770 - val_loss: 0.1249
Epoch 2/30
438/438 [==============================] - 3s 8ms/step - loss: 0.0922 - val_loss: 0.0934
Epoch 3/30
438/438 [==============================] - 3s 6ms/step - loss: 0.0879 - val_loss: 0.0861
Epoch 4/30
438/438 [==============================] - 3s 6ms/step - loss: 0.0868 - val_loss: 0.0857
Epoch 5/30
438/438 [==============================] - 3s 6ms/step - loss: 0.0864 - val_loss: 0.0851
Epoch 6/30
438/438 [==============================] - 2s 5ms/step - loss: 0.0862 - val_loss: 0.0847
Epoch 7/30
438/438 [==============================] - 3s 8ms/step - loss: 0.0860 - val_loss: 0.0846
Epoch 8/30
438/438 [==============================] - 3s 6ms/step - loss: 0.0858 - val_loss: 0.0844
Epoch 9/30
438/438 [==============================] - 3s 6ms/step - loss: 0.0856 - val_loss: 0.0844
Epoch 10/30
438/438 [==============================] - 3s 6ms/step - loss: 0.0854 - val_loss: 0.0844

In [ ]:
def get_recommendations(product_id, product_df, all_embeddings, top_k=10):
    # Find the index of the product
    idx = product_df.index[product_df['uniq_id'] == product_id].tolist()[0]

    # Compute cosine similarity
    cosine_sim = cosine_similarity(all_embeddings[idx:idx+1], all_embeddings)[0]

    # Get top_k similar products
    sim_scores = list(enumerate(cosine_sim))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_k+1]  # Exclude the target product itself

    product_indices = [i[0] for i in sim_scores]
    return product_df.iloc[product_indices]

# Example usage
product_id = 'c2d766ca982eca8304150849735ffef9'
recommended_products = get_recommendations(product_id, product_df, cls_embeddings)
print(recommended_products[['uniq_id', 'product_name', 'product_url']])

In [11]:
# def get_recommendations_by_substring(substring, product_df, all_embeddings, top_k=10):
#     # Filter product DataFrame based on the substring in the product name
#     filtered_df = product_df[product_df['product_name'].str.contains(substring, case=False, na=False)]

#     if len(filtered_df) == 0:
#         print("No products found with the specified substring.")
#         return None

#     # Compute embeddings for filtered products
#     filtered_embeddings = all_embeddings[filtered_df.index]

#     # Compute cosine similarity
#     cosine_sim = cosine_similarity(filtered_embeddings, all_embeddings)

#     # Get top_k similar products
#     sim_scores = []
#     for idx, row in filtered_df.iterrows():
#         cosine_similarities = cosine_sim[idx]
#         for i, sim in enumerate(cosine_similarities):
#             sim_scores.append((i, sim))

#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[:top_k]

#     product_indices = [i[0] for i in sim_scores]
#     return filtered_df.iloc[product_indices]

# # Example usage
# substring = "kitchen"
# recommended_products = get_recommendations_by_substring(substring, product_df, cls_embeddings)
# if recommended_products is not None:
#     print(recommended_products[['uniq_id', 'product_name', 'product_url']])

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got Index([ 1492,  3772,  4489,  8144,  8145,  8146,  8149,  8155,  8156,  8157,
        8160,  8164,  8165, 10068, 10071, 10073, 13712, 13713, 18393, 18421,
       18431, 18456, 18464, 18467, 18471, 18563, 18629, 18959, 19058, 19249,
       19314, 19322, 19378],
      dtype='int64')

In [ ]:
# def get_recommendations(product_name_substr, df=product_df, cosine_sim=cosine_sim, category=None):
#     # Filter products whose names contain the provided substring
#     name_filtered_df = df[df['product_name'].str.contains(product_name_substr, case=False, na=False)]

#     # Check if any products match the substring
#     if name_filtered_df.empty:
#         return f"No products found with name containing '{product_name_substr}'"

#     # Get the index of the first matching product
#     idx = name_filtered_df.index[0]

#     # Calculate similarity scores
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Optionally filter by category if provided
#     if category:
#         category_products = df[df['product_category_tree'].str.contains(category, case=False, na=False)]
#         category_indices = category_products.index.tolist()

#         # Filter similarity scores by category
#         sim_scores = [score for score in sim_scores if score[0] in category_indices]

#     # Get top 15 recommendations excluding the product itself
#     sim_scores = sim_scores[:10]
#     product_indices = [i[0] for i in sim_scores]

#     return df[['uniq_id', 'product_name', 'product_url']].iloc[product_indices]

# # Example usage:
# recommended_products = get_recommendations('Handmade')
# for index, row in recommended_products.iterrows():
#     print(f"Product ID: {row['uniq_id']}")
#     print(f"Product Name: {row['product_name']}")
#     print(f"Product link: {row['product_url']}")

In [ ]:
# num_products = product_df.shape[0]
# y_train = np.random.rand(num_products, 50)
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# model.fit(text_vector, y_train, epochs=10, batch_size=64, validation_split=0.5, callbacks = [early_stopping])

In [ ]:
# # Mendapatkan embedding dari teks
# text_embeddings = model.predict(text_vector)

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

# # Menghitung kemiripan kata dengan model
# cosine_sim = cosine_similarity(text_embeddings, text_embeddings)
# print(cosine_sim)

In [ ]:
# Fungsi untuk mendapatkan rekomendasi
# def get_recommendations(product_id, category, df=product_df, cosine_sim=cosine_sim):
#     try:
#         idx = df.index[df['uniq_id'] == product_id].tolist()[0]
#     except IndexError:
#         return f"Product ID {product_id} not found."

#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Filter by category
#     category_products = df[df['product_category_tree'].str.contains(category, case=False, na=False)]
#     category_indices = category_products.index.tolist()

#     # Get top 15 recommendations excluding the product itself and filtering by category
#     sim_scores = [score for score in sim_scores if score[0] in category_indices and score[0] != idx]
#     sim_scores = sim_scores[:10]
#     product_indices = [i[0] for i in sim_scores]

#     return df[['product_name', 'product_url']].iloc[product_indices]

# # Contoh penggunaan
# recommended_products = get_recommendations('c2d766ca982eca8304150849735ffef9', 'Clothing')
# print(recommended_products)

In [ ]:
# from IPython.display import display, Image

# def get_recommendations(product_id, category, df=product_df, cosine_sim=cosine_sim):
#     try:
#         idx = df.index[df['uniq_id'] == product_id].tolist()[0]
#     except IndexError:
#         return f"Product ID {product_id} not found."

#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Filter by category
#     category_products = df[df['product_category_tree'].str.contains(category, case=False, na=False)]
#     category_indices = category_products.index.tolist()

#     # Get top 15 recommendations excluding the product itself and filtering by category
#     sim_scores = [score for score in sim_scores if score[0] in category_indices and score[0] != idx]
#     sim_scores = sim_scores[:15]
#     product_indices = [i[0] for i in sim_scores]

#     return df[['uniq_id', 'product_name', 'product_url']].iloc[product_indices]


# recommended_products = get_recommendations('7f7036a6d550aaa89d34c77bd39a5e48', 'Furniture')

# for index, row in recommended_products.iterrows():
#     print(f"Product ID: {row['uniq_id']}")
#     print(f"Product Name: {row['product_name']}")
#     print(f"Product link: {row['product_url']}")

In [ ]:
# print(product_df[['uniq_id', 'product_name']].head(30))

# kitchen_products = product_df[product_df['product_name'].str.contains("bowl", case=False, na=False)]

# # Display the first 10 rows of the filtered DataFrame showing `uniq_id` and `product_name`
# print(kitchen_products[['uniq_id', 'product_name']].head(30))

In [12]:
from tensorflow import lite

model = tf.keras.models.load_model('recommender')

# Convert the model to TFLite format
converter = lite.TFLiteConverter.from_keras_model(model)

# Set the target specification to include select TensorFlow ops
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # TFLite built-in ops
    tf.lite.OpsSet.SELECT_TF_OPS     # Select TensorFlow ops
]

# Disable the experimental lowering of tensor list ops
converter._experimental_lower_tensor_list_ops = False

tflite_model = converter.convert()

# # Save the TFLite model to a fil
# open('recommender.tflite', 'wb').write(tflite_model)

# Save the TFLite model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)